In [4]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

In [5]:
# CHECK WETHER CUDA IS AVAILABLE OR NOT. IF AVAILABLE, ASSIGN DEVICE AS CUDA
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [6]:
#TRANSFORMS ON THE DATASET, SET NORMALISATION FOR EACH IMAGE AND LOAD DATASET
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.FashionMNIST(root='./data', train=True, transform=trans, download=True)
test_set = dset.FashionMNIST(root='./data', train=False, transform=trans, download=True)


Processing...
Done!


In [7]:
batch_size = 70

train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False)

## Model Initialisation:- 

Generated a model that takes in each image and generates an output with processing on 3 convolution layers followed by 2 fully connected layers

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
model=Net().to(device)

In [15]:
print(device)
Net()

cuda


Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [16]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criterion = nn.CrossEntropyLoss()
iter = 0

In [ ]:
count=0
avg_accuracy=0
for epoch in range(50):
    # trainning
    ave_loss = 0
    for i, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        x, target = Variable(x).to(device), Variable(target).to(device)
        out = model(x)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        iter+=1
        #print ("==>>> epoch: {}, batch index: {}, train loss: {:.6f}".format(epoch, batch_idx+1, loss.data[0]))
        if i%500 ==0 or (i+1) == len(train_loader):
            correct=0
            total=0
            for testidx, (x,target) in enumerate(test_loader):
                x = Variable(x, volatile=True).to(device)
                outputs = model(x)
                _, predicted=torch.max(outputs.data, 1)
                total+=target.size(0)
                correct+=(predicted==target.cuda()).sum()
                accuracy=(100*correct/total).float()
                count+=1
                avg_accuracy+=accuracy
               # print("iteration {}, loss{}, accuracy{:.3f}".format(iter, loss.data[0], accuracy))
avg_accuracy/=count
print("average accuracy is {}".format(avg_accuracy))
        

/home/yash/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
